In [ ]:
from azureml.core import Workspace, Experiment

# Configure experiment
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name='cifar10_cnn_as_mount')

In [ ]:
from azureml.core import Dataset

ds = Dataset.get_by_name(ws, name='cifar10')

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# cluster configuration
cluster_name = "mldemocompute"
min_nodes = 0
max_nodes = 2
vm_size = "STANDARD_D2_V2"

# verify that the cluster exists already
try:
    aml_cluster = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    print('Cluster not `%s` not found, creating one now.' % cluster_name)
    config = AmlCompute.provisioning_configuration(vm_size=vm_size, min_nodes=min_nodes, max_nodes=max_nodes)
    aml_cluster = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=config)

# wait until the cluster is ready
aml_cluster.wait_for_completion(show_output=True)

In [ ]:
import os
from azureml.core import Environment

# define the PIP requirements file location
pipreq = 'requirements.txt'
pipreq_path = os.path.join(os.getcwd(), '.azureml', pipreq)

# set a remote environment from a PIP requirement file
myenv = Environment.from_pip_requirements(name = "remote_env", file_path = pipreq_path)
# show environment object
myenv

In [ ]:
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory="code",
                      script='cifar10_cnn_remote.py',
                      arguments=['--input', ds.as_named_input('cifar10').as_mount()],
                      ompute_target = aml_cluster,
                      environment = myenv)

run = exp.submit(src)
RunDetails(run).show()